Design a feedforward neural network from scratch for following neural network.
A) Binary Classication
B)Multiclass Classication(4 input Neuron, 3 neuron in rst hidden layer, 4 neuron in second hidden layer,3 neuron in output layer)

Generate dataset using make_blobs,make_moons,make_circles. Use sigmoid activation function for all layer in Binary Classication. Sigmoid at
hidden layer and softmax at output layer for multiclass classication. Use mean squared error loss function for binary classication and binary
cross entropy loss function for multiclass classication.

A) Binary Classication

In [ ]:
import numpy as np
from sklearn.datasets import make_blobs, make_moons, make_circles
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Sigmoid activation function
def sigmoid(x):
 return 1 / (1 + np.exp(-x))
def sigmoid_derivative(x):  #
 return sigmoid(x) * (1 - sigmoid(x))

# Softmax activation function
def softmax(x): #multiclass
 exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
 return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# Loss functions
def mean_squared_error(y_true, y_pred):
 return np.mean((y_true - y_pred) ** 2)
def binary_cross_entropy(y_true, y_pred):
 return -np.mean(y_true * np.log(y_pred + 1e-9))

In [ ]:
# Feedforward Neural Network class
class FeedforwardNeuralNetwork:
    def __init__(self, input_dim, hidden_dim, output_dim, binary_classification=True):
        self.binary_classification = binary_classification

        # Initialize weights and biases
        self.W1 = np.random.randn(input_dim, hidden_dim)
        self.b1 = np.zeros((1, hidden_dim))
        self.W2 = np.random.randn(hidden_dim, output_dim)
        self.b2 = np.zeros((1, output_dim))

    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = sigmoid(self.z1)

        # Hidden to output layer
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        if self.binary_classification:
            self.a2 = sigmoid(self.z2)  # Sigmoid for binary
        else:
            self.a2 = softmax(self.z2)  # Softmax for multiclass
        return self.a2

    def backward(self, X, y, y_pred, lr):
        # Output layer error
        if self.binary_classification:
            d_z2 = (y_pred - y) * sigmoid_derivative(self.z2)
        else:
            d_z2 = y_pred - y
        d_W2 = np.dot(self.a1.T, d_z2)
        d_b2 = np.sum(d_z2, axis=0, keepdims=True)

        # Hidden layer error
        d_a1 = np.dot(d_z2, self.W2.T)
        d_z1 = d_a1 * sigmoid_derivative(self.z1)
        d_W1 = np.dot(X.T, d_z1)
        d_b1 = np.sum(d_z1, axis=0, keepdims=True)

        # Update weights and biases
        self.W2 -= lr * d_W2
        self.b2 -= lr * d_b2
        self.W1 -= lr * d_W1
        self.b1 -= lr * d_b1

    def train(self, X, y, epochs, lr): # Define the train method within the class
        for epoch in range(epochs):
            y_pred = self.forward(X)
            if self.binary_classification:
                loss = mean_squared_error(y, y_pred)
            else:
                loss = binary_cross_entropy(y, y_pred)
            self.backward(X, y, y_pred, lr)
            if (epoch + 1) % 100 == 0:
                print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}")

In [ ]:
# Initialize and train binary classification model
binary_model = FeedforwardNeuralNetwork(input_dim=2, hidden_dim=5, output_dim=1, binary_classification=True)
binary_model.train(X_train, y_train, epochs=1000, lr=0.01)

Epoch 100/1000, Loss: 0.0010
Epoch 200/1000, Loss: 0.0005
Epoch 300/1000, Loss: 0.0003
Epoch 400/1000, Loss: 0.0002
Epoch 500/1000, Loss: 0.0002
Epoch 600/1000, Loss: 0.0002
Epoch 700/1000, Loss: 0.0001
Epoch 800/1000, Loss: 0.0001
Epoch 900/1000, Loss: 0.0001
Epoch 1000/1000, Loss: 0.0001


In [ ]:
# Test model
y_pred_test = binary_model.forward(X_test)
print("Test Loss:", mean_squared_error(y_test, y_pred_test))

Test Loss: 8.575005180900871e-05


B) Multiclass Classication

In [ ]:
# Import necessary libraries
import numpy as np
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from scipy.special import softmax


In [ ]:
# Generate dataset using make_blobs for multiclass classification
X, y = make_blobs(n_samples=300, centers=3, n_features=4, random_state=42)

# One-hot encode the labels
encoder = OneHotEncoder(sparse_output=False) # Updated argument
y_onehot = encoder.fit_transform(y.reshape(-1, 1))

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)


In [ ]:
# Sigmoid activation function
def sigmoid(z):
 return 1 / (1 + np.exp(-z))

# Derivative of sigmoid for backpropagation
def sigmoid_derivative(z):
 return sigmoid(z) * (1 - sigmoid(z))

In [ ]:
# Initialize weights and biases
def initialize_parameters(input_size, hidden1_size, hidden2_size, output_size):
  np.random.seed(42)
  W1 = np.random.randn(input_size, hidden1_size) * 0.01
  b1 = np.zeros((1, hidden1_size))
  W2 = np.random.randn(hidden1_size, hidden2_size) * 0.01
  b2 = np.zeros((1, hidden2_size))
  W3 = np.random.randn(hidden2_size, output_size) * 0.01
  b3 = np.zeros((1, output_size))
  return W1, b1, W2, b2, W3, b3

In [ ]:
# Forward propagation
def forward_propagation(X, W1, b1, W2, b2, W3, b3):
 Z1 = np.dot(X, W1) + b1
 A1 = sigmoid(Z1)
 Z2 = np.dot(A1, W2) + b2
 A2 = sigmoid(Z2)
 Z3 = np.dot(A2, W3) + b3
 A3 = softmax(Z3, axis=1)
 return Z1, A1, Z2, A2, Z3, A3

In [ ]:
# Compute categorical cross-entropy loss
def compute_loss(y_true, y_pred):
 m = y_true.shape[0]
 loss = -np.sum(y_true * np.log(y_pred + 1e-15)) / m

In [ ]:
# Backpropagation
def backpropagation(X, y, Z1, A1, Z2, A2, Z3, A3, W1, W2, W3):
 m = X.shape[0]
 dZ3 = A3 - y
 dW3 = np.dot(A2.T, dZ3) / m
 db3 = np.sum(dZ3, axis=0, keepdims=True) / m
 dZ2 = np.dot(dZ3, W3.T) * sigmoid_derivative(Z2)
 dW2 = np.dot(A1.T, dZ2) / m
 db2 = np.sum(dZ2, axis=0, keepdims=True) / m
 dZ1 = np.dot(dZ2, W2.T) * sigmoid_derivative(Z1)
 dW1 = np.dot(X.T, dZ1) / m
 db1 = np.sum(dZ1, axis=0, keepdims=True) / m
 return dW1, db1, dW2, db2, dW3, db3


In [ ]:
# Update parameters
def update_parameters(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, learning_rate):
 W1 -= learning_rate * dW1
 b1 -= learning_rate * db1
 W2 -= learning_rate * dW2
 b2 -= learning_rate * db2
 W3 -= learning_rate * dW3
 b3 -= learning_rate * db3
 return W1, b1, W2, b2, W3, b3

In [ ]:
# Train the neural network
def train(X, y, input_size, hidden1_size, hidden2_size, output_size, epochs, learning_rate):
 W1, b1, W2, b2, W3, b3 = initialize_parameters(input_size, hidden1_size, hidden2_size, output_size)
 for epoch in range(epochs):
  Z1, A1, Z2, A2, Z3, A3 = forward_propagation(X, W1, b1, W2, b2, W3, b3)
  loss = compute_loss(y, A3)
  dW1, db1, dW2, db2, dW3, db3 = backpropagation(X, y, Z1, A1, Z2, A2, Z3, A3, W1, W2, W3)
  W1, b1, W2, b2, W3, b3 = update_parameters(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, learning_rate)
 if epoch % 100 == 0:
  print(f"Epoch {epoch}, Loss: {loss}")
 return W1, b1, W2, b2, W3, b3


In [ ]:
# Model training
input_size = 4
hidden1_size = 3
hidden2_size = 4
output_size = 3
epochs = 1000
learning_rate = 0.1
W1, b1, W2, b2, W3, b3 = train(X_train, y_train, input_size, hidden1_size, hidden2_size, output_size, epochs, learning_rate)


In [ ]:
# Test the model
_, _, _, _, _, A3_test = forward_propagation(X_test, W1, b1, W2, b2, W3, b3)
test_loss = compute_loss(y_test, A3_test)
print(f"Test Loss: {test_loss}")


Test Loss: None


In [ ]:
# Evaluate accuracy
predictions = np.argmax(A3_test, axis=1)
y_true = np.argmax(y_test, axis=1)
accuracy = np.mean(predictions == y_true)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 30.00%
